Import libraries

In [11]:
import tensorflow
import numpy as np
import cv2
from tkinter import *
from tkinter import filedialog

Load Trained model

In [12]:
loaded_model = tensorflow.keras.models.load_model('face_model')

Model prediction implementation

In [13]:
cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: "Angry", 1:"Disgusted", 2:"Fearful", 3:"Happy", 4: "Neutral", 5:"Sad", 6:"Surprised"} #array of emotion types
#Video to prediction implementation
def Video(file_path):
    cap = cv2.VideoCapture(file_path) #apply path to video capture
    if (cap.isOpened() == False): #check video found state
        print("video file not found")
        exit()

    while (cap.isOpened()):
        ret, frame = cap.read() #read video frames
        if not ret:
            break
        bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') #boundry box implement by Cascade Classifier
        gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)# frame turn into grayscale
        num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors= 5) # detect MultiScale
        for(x,y,w,h) in num_faces:
            cv2.rectangle(frame, (x,y-50), (x+w, y+h+10), (255,0,0),2) #apply rectangle to detection area.
            roi_gray_frame = gray_frame[y:y+h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)), -1),0) # crop the image  size
            emotion_prediction = loaded_model.predict(cropped_img) #apply prediction result to crop image
            maxindex = int(np.argmax(emotion_prediction))#covert predication into integers
            cv2.putText(frame,emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,255),2,cv2.LINE_AA) #put text in to top of the rectangle
        cv2.imshow('Video  - Press q to close',cv2.resize(frame,(480,280), interpolation= cv2.INTER_CUBIC))#display the result
        if cv2.waitKey(1) & 0xFF == ord('q'): #apply exit function
                cap.release()
                cv2.destroyAllWindows()
                break


In [15]:
def browseFiles(): #path selection by  gui in windows explore
    filename = filedialog.askopenfilename(initialdir = "/",
                                          title = "Select a File",
                                          filetypes = (("Video File",
                                                        "*.mp4*"),
                                                       ("all files",
                                                        "*.*")))
    Video(filename) #run video prediction system
    #create GUI  window
if __name__ == '__main__':
    window = Tk()
    window.title("File Explorer")
    window.geometry("500x500")
    window.config(background="white")
    label_file_explorer = Label(window,
                            text = "File Explorer using Tkinter",
                            width = 100, height = 4,
                            fg = "blue")
    button_explore = Button(window,
                        text = "Browse Files",
                        command = browseFiles)
    label_file_explorer.grid(column = 1, row = 1)
    button_explore.grid(column = 1, row = 2)
    window.mainloop()